In [1]:
#coding: utf-8
#Using python's PIL module to process images 
from PIL import Image

In [2]:
class node: #Node's class
    #Define node construction method
    def __init__(self,right=None,left=None, parent=None, weight=0, code=None):
        self.left = left 
        self.right = right 
        self.parent = parent 
        self.weight = weight #weight
        self.code = code #Node value

In [3]:
#Define function, convert the color image to a gray image, at this time each pixel of the image can be represented by a separate pixel value
def picture_convert(filename,newfilename): 
    picture = Image.open(filename)
    picture = picture.convert('L')#Convert bmp picture to gray value picture
    picture. save(newfilename)#Save grayscale image
    return picture

In [4]:
#Define function, count the number of times each pixel appears
def pixel_number_caculate(list):
    pixel_number={}
    for i in list:
        if i not in pixel_number. keys():
            pixel_number[i]=1 #If the pixel is not in the character frequency dictionary, add it directly
        else:
            pixel_number[i] += 1 #If it exists in the character frequency string dictionary, add one to the corresponding value 
    return pixel_number

In [5]:
#Construct a node, give its value and corresponding weight respectively 
def node_construct(pixel_number): 
    node_list =[]
    for i in range(len(pixel_number)):
        node_list.append(node(weight=pixel_number[i][1],code=str(pixel_number[i][0])))
    return node_list

In [6]:
#According to the list of leaf nodes, generate the corresponding Huffman coding tree
def tree_construct(listnode):
    listnode = sorted(listnode,key=lambda node:node.weight) 
    while len(listnode) != 1:
        # Each time the two pixel points of the weighted value are merged
        low_node0,low_node1 = listnode[0], listnode[1]
        new_change_node = node()
        new_change_node.weight = low_node0.weight + low_node1.weight
        new_change_node.left = low_node0
        new_change_node.right = low_node1
        low_node0.parent = new_change_node
        low_node1.parent = new_change_node
        listnode.remove(low_node0)
        listnode.remove(low_node1)
        listnode.append(new_change_node)
        listnode = sorted(listnode, key=lambda node:node.weight) 
    return listnode

In [7]:
def _to_Bytes(data):
    b = bytearray()
    for i in range(0, len(data), 8):
        b.append(int(data[i:i+8], 2))
    return bytes(b)

In [8]:
#The main function of Huffman coding, which completes the coding of pixels by calling other functions
import array
def Huffman_Coding(picture):
    #Get the width and height of the picture
    width = picture.size[0]
    height = picture.size[1]
    im = picture.load()
    print ("Grayscale image width is"+str(width)+"pixel")
    print ("Grayscale image height is"+str(height)+"pixel")
 
    #Save the pixels in Jist, the original two-dimensional matrix becomes a one-dimensional array 
    list =[]
    for i in range(width):
        for j in range(height): 
            list.append(im[i,j])
    #Count the number of times of each pixel, and sort according to the number of occurrences from small to large 
    #this function returns a dictionary
    pixel_number = pixel_number_caculate(list)
    pixel_number = sorted(pixel_number.items(),key=lambda item:item[1])
           
    #Construct the node list according to the value of the pixel and the number of occurrences 
    #create node for each pixel
    node_list = node_construct(pixel_number)
    # Construct a Huffman tree, save the head node
    #function returns last element in the nodes list 
    head = tree_construct(node_list)[0]
           
    #Construction code table
    coding_table = {}
    for e in node_list:
        new_change_node = e
        coding_table.setdefault(e.code,"")
        #traverse the tree from bottom to top
        while new_change_node !=head:
            #check if it is on the left 
            if new_change_node.parent.left == new_change_node: 
                coding_table[e.code] = "1" + coding_table[e.code] 
            else:
                coding_table[e.code] = "0" + coding_table[e.code] 
            new_change_node = new_change_node.parent
            
    #Output the gray value and code of each image accumulation point
    for key in coding_table.keys():
        print ("source pixel" + key + "code strength after encoding:" + coding_table[key])
 
    #Output encoding table
    print ("The coding table is:", coding_table)
 
    #Convert the encoding result of the image into a string and save it in bin
    coding_result = ''
    data = array.array('B')  # create array of bytes.
    for i in range(width):
        for j in range(height):
            for key,values in coding_table.items(): 
                if str(im[i,j]) == key:
                    coding_result = coding_result+values
                    break
    with open('test.bin', 'wb') as f:
        f.write(_to_Bytes(coding_result))
    return coding_table

In [9]:
import os
#Restore the original bmp image, traverse the result of Huffman encoding, for each traversed character is in the codeword list 
#Search in #, if not found, add the next character, continue to search, repeat this step until you find it in the code word list 
#Go to the pixel corresponding to the codeword, and put the pixel value corresponding to the codeword into the pixel list.
def Decoding(width,height,coding_table,coding_result,path): 
    code_read_now = ""#The currently read code 
    new_pixel =[] 
    i = 0
    while (i != coding_result.__len__()):
        #Read one letter each time
        code_read_now = code_read_now + coding_result[i]
        for key in coding_table.keys():
            #If the currently read code exists in the code table 
            if code_read_now == coding_table[key]: 
                new_pixel.append(key) 
                code_read_now = ""
                break
        i +=1
    #Construct a new image
    decode_image = Image.new( 'L' ,(width,height)) 
    k = 0 
    # 
    print(len(new_pixel))
    for i in range(width):
        for j in range(height):
            decode_image.putpixel((i,j),(int(new_pixel[k]))) 
            k += 1
    decode_image.save('decode.bmp') 
    print("Decoding has been completed: the picture is stored as decode.bmp")

In [10]:
#file = r'D:\Online Courses Certificates\ITI AI Pro\10-Algorithmic Workshop\Project\colored.bmp'
file = r'D:\Online Courses Certificates\ITI AI Pro\10-Algorithmic Workshop\Project\colored.bmp'
picture = picture_convert(file,'new.bmp')
width = picture.size[0]
height = picture.size[1]
coding_table = Huffman_Coding(picture)

Grayscale image width is612pixel
Grayscale image height is408pixel
source pixel2code strength after encoding:100110100010000111
source pixel232code strength after encoding:100110100010000110
source pixel231code strength after encoding:100110100010000101
source pixel223code strength after encoding:100110100010000100
source pixel5code strength after encoding:11000110011011111
source pixel219code strength after encoding:11000110011011110
source pixel3code strength after encoding:10011010001001011
source pixel224code strength after encoding:10011010001001010
source pixel220code strength after encoding:10011010001001001
source pixel217code strength after encoding:10011010001001000
source pixel8code strength after encoding:1100011001101110
source pixel6code strength after encoding:1001101000110011
source pixel221code strength after encoding:1001101000110010
source pixel226code strength after encoding:1001101000100111
source pixel7code strength after encoding:1001101000100110
source pixel225c

In [11]:
import numpy as np
coding_result = open('test.bin','rb').read()
b = bytearray()
b = np.fromfile('test.bin', dtype = "uint8")
h = np.unpackbits(b)
#print(h)
coding_string = ""
for binary_value in h:
    coding_string += str(binary_value)
#print(coding_string)
#print(' '.join(str(c) for c in coding_result))
path = r'C:\Documents\test.bin'
Decoding(width,height,coding_table,coding_string,path)

249696
Decoding has been completed: the picture is stored as decode.bmp
